# 머신러닝 미리보기

In [9]:
import pandas as pd
from konlpy.tag import Okt
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


## 1. LDA(Latent Dirichlet Analysis)

### 1) 데이터 불러오기

In [3]:
reply_data_set = pd.read_csv("data/appreply2.csv", index_col = 0)
reply_data_set

,text,score
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3
...,...,...
995,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1
996,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3
997,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3
998,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1


### 2) 형태소 분석기

In [ ]:
okt = Okt()

# 각 문장을 정규표현식으로 전처리한 후, 형태소 분석기를 통해 문장별로 담아라

stopwords = []
sum_text_list = []

for text in reply_data_set["text"] :
    word_list = []
    text = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", text)
    text_list = okt.pos(text, stem = True)
    
    for tex, pos in text_list :
        if (pos in ['Noun', 'Verb']) & (tex not in stopwords) :
            word_list.append(tex)
    
    sum_text_list.append(" ".join(word_list))

sum_text_list
    

['배달 민족 리뷰 자주 참고 편입 한가지 건의 사항 최신 순 점 뿐 제 주문 자하 메뉴 메뉴 리뷰 확인 하다 기능 메뉴 검색 기능 리뷰 특정 메뉴 검색 기능 문수 않다 메뉴 리뷰 보기 위해 드 래그 하다 시간 소요 되다 비 효율 발생 하다 긍정 검토 해 주심',
 '내 주문 하다 과거 목록 검색 기능 분명 가게 시키다 기억 뭘 시키다 먹다 맞다 알 싶다 찾기 먹다 메뉴 검색 를 곱창 치면 과거 시키다 곱창 목록 뜨다 가게 리뷰 창 리뷰 보기',
 '검색 화면 전체 배달 포장 탭 배달 탭을스 크롤 내리다 볼때 아래 로스 크롤 하다 자꾸만 왼쪽 오른쪽 넘어가다 전체 탭 포장 탭 보다 되다 정말 검색 후 배달 탭 포장 탭 마트 탭뭐 하나 선택 하다 좌우 넘기다 하다 가끔 도아 고스 크롤 한번 내리다 때 왼쪽 넘어가다 전체 탭 가게 되다 쓰다',
 '배달 팁 순 정렬 지역 따르다 추가 배달 료 별도 체크 되어다 배달 팁 하나 누르다 가격 대별 금액 체크 배달 팁 도로 받다 확인 해 야합',
 '최근 업데이트 안드로이드 정도 배민 어플 실행 업데이트 나오다 업데이트 누르다 업데이트 진행 열기 나오다 열기 누르다 업데이트 나오다 무한 반복 삭제 재 설치 환경설정 등 해보다 증상 요 다른 도 증상 보이다 글 보다 이번 업데이트 관련 파일 확인 보이다 참고 사양 폰 자다 되다 보다 핸드폰 사양 문제 이번 업데이트 보이다',
 '찌다 장구 좀 해주다 대체 언제 이면 개선 되다 요 독점 해이 건가 앱 결제 관련 부분 정말 로고 앱 구성 정말 배민 페이 사용 절차 쓰다 것 찌다 놓다 매장 한식 양식 중식 카페 업종 별로 구분 좀 시키다 순 찜 매장 개도 이름 전부 외우다 매번 찾기 서재 검색 거나 주문 하다 목록 찾다 들어가다',
 '앱 실행 시 로그인 여부 먼저 물어보다 비회 이용 선택 하다 경우 나중 입력 선택 후주 결제 시 받다 주소 설정 하다 순서 변경 요 저 때 어플 설치 하다 삭제 설치 자주 펴다 달다 고재 설치 실행 회원정보 주소 저장 을해 놨음 어플 실행 로그인 

### 3) CountVectorizer

In [ ]:
vectorizer = CountVectorizer(
    max_df = 0.1,               # 전체 단어 등장 비율이 p 이상인 것만 사용할 것
    min_df = 2,                 # 단어가 적어도 n개 이상인 것만 사용할 것
    max_features = 1000,        # 상위 항목 n개만 사용할 것
    ngram_range = (1,2)         # 단어의 조합
)

In [7]:
feature_vec = vectorizer.fit_transform(sum_text_list)
feature_vec.shape

(1000, 1000)

In [8]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

['가가' '가게 검색' '가게 들어가다' '가게 리뷰' '가게 메뉴' '가게 목록' '가게 배달' '가게 별로' '가게 전화'
 '가게 주문' '가격' '가기' '가끔' '가나' '가능' '가다' '가맹' '가면' '가안' '가요' '가원' '가입'
 '가입 하다' '가장' '가족' '가지' '가하다' '갈다' '갈수록' '갈아타다' '감사' '감사하다' '갑자기' '강제'
 '갖다' '개다' '개발' '개발자' '개선' '개선 되다' '개선 되어다' '개선 하다' '개선 해주다' '개인' '개편'
 '갤럭시' '거기' '거나' '거리' '거의' '거지' '건가' '건물' '건의' '건의 사항' '건지다' '건지다 모르다'
 '걸다' '걸리다' '검색 결과' '검색 기능' '검색 되다' '검색 하다' '검토' '겁니다' '겠다' '겪다' '결과' '결재'
 '결제 되다' '결제 수단' '결제 하다' '경우' '경험' '계속' '계정' '고객' '고객 센터' '고려' '고르다' '고민'
 '고배' '고스' '고스 크롤' '고요' '고재' '고치다' '고하' '고해' '공지' '과거' '과정' '관련' '관리' '광고'
 '구분' '구성' '굳이' '그거' '그것' '그게' '그냥' '그대로' '그리다' '근처' '글자' '금액' '금은' '기간'
 '기기' '기능 리뷰' '기능 만들다' '기능 생기다' '기능 추가' '기다' '기다리다' '기도' '기본' '기분' '기사'
 '기사 배달' '기억' '기업' '기요' '기입' '기전' '기존' '기준' '까먹다' '깔다' '깜빡' '꾸다' '끄다' '끊다'
 '끝나다' '끼다' '나가다' '나다' '나서다' '나서다 결제' '나오다' '나중' '남기다' '남다' '내다' '내려가다'
 '내리다' '내용' '넘기다' '넘다' '넘어가다' '넘어가다 않다' '넣다' '네이버' '네이버 로그인' '네이버 아이디'
 '네트워크' '노력' '노출' '노출 되다' '노트' '놓다' '누가' '누구' '누락' '누르다' '누

### 4) LDA

In [10]:
lda = LatentDirichletAllocation(n_components = 5)
lda.fit(feature_vec)

,n_components,5
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [11]:
print(lda.components_.shape)
print(lda.components_)

(5, 1000)
[[ 2.15612717  0.20767643  0.20038908 ...  4.01416897  0.20067417
   0.20069326]
 [ 3.90362739  3.01944629  3.94003773 ...  0.20000442  4.28617894
   0.20000166]
 [ 0.2011034   3.22519181  2.45206848 ...  0.20329404  2.04942552
   0.20007244]
 [ 1.53852004  0.20000437  0.20000617 ...  0.20057114 15.12257452
   8.19901684]
 [ 0.200622    3.3476811   0.20749854 ...  3.38196143  0.34114685
   0.2002158 ]]
